In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-141247",  resource_group="aml-quickstarts-141247", subscription_id="f5091c60-1c3c-430f-8d81-d802f6bf2414")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A22SBTVYQ to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141247
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-141247


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azureml.exceptions import ComputeTargetException

cpu_cluster_name = "project1-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C': uniform(0.1, 2), '--max_iter': uniform(500, 5000)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn('.', compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, policy=policy, estimator=est, 
                                    primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=15)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hper_run = exp.submit(config=hyperdrive_config)

In [5]:
from azureml.widgets import RunDetails
RunDetails(hper_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.

best_run = hper_run.get_best_run_by_primary_metric()

In [10]:
mod = best_run.register_model(model_name='bank-marketing-log-reg', model_path="/outputs/bank-marketing-log-reg.joblib", 
        tags={"type": "logistic regression", "dataset":"bank marketing"})

In [11]:
best_run.download_file("/outputs/bank-marketing-log-reg.joblib", "outputs/best-hyperdrive-mod.joblib")

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory().from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
print('cleaning')
x, y = clean_data(data)
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, random_state=666)

print('train data to csv')
train = pd.concat([x_train, y_train], axis=1)
train.to_csv("training/train_banking.csv", index=False)

print('upload to datastore')
ds = ws.get_default_datastore()
ds.upload(src_dir="./training", target_path="bankmarketing", overwrite=True, show_progress=True)

cleaning
train data to csv
upload to datastore
Uploading an estimated of 1 files
Uploading ./training/train_banking.csv
Uploaded ./training/train_banking.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_31822e1b0cb147e595c7a4200bde858b

In [13]:
print('get dataset reference')
train_data = TabularDatasetFactory().from_delimited_files(path=ds.path('bankmarketing/train_banking.csv'))

get dataset reference


In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5, 
    compute_target=cpu_cluster, 
    blocked_models=['XGBoostClassifier'])

In [15]:
# Submit your automl run
automl_exp = Experiment(ws, name="automl-run")

In [16]:
auto_run = automl_exp.submit(automl_config, show_output=False)

Running on remote.


In [17]:
RunDetails(auto_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [19]:
# Retrieve and save your best automl model.
best_auto_run, fitted_model = auto_run.get_output()
best_auto_run.register_model(model_name="bank_marketing-automml", model_path="./outputs/")

Model(workspace=Workspace.create(name='quick-starts-ws-141247', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-141247'), name=bank_marketing-automml, id=bank_marketing-automml:1, version=1, tags={}, properties={})

In [20]:
#Delete cluster
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

